In [14]:
from google.cloud import bigquery

import pandas as pd
import numpy as np
from datetime import date,datetime,timedelta,timezone

import math
import os

import tensorflow as tf
import tensorflow_decision_forests as tfdf  # constantly registered to load model 
print(tf.__version__)
print(tfdf.__version__)

2.12.0
1.4.0


In [15]:
# import functions_framework
# @functions_framework.http
# def xgb_tf_predict_incident_severity(request):

# Load Configuration Data and Constant Variable

In [16]:
projectId="pongthorn"
#_model='binary_xgb_tf_model'   # the is resonable because unsseen and tune 70%
all_prediction=True
is_evaluation=True

_model="demo_binary_xgb_tf_model"  # best 75% on unseen test
model_version=f'{_model}_demo_t150723'

model_gs_path=f"gs://demo-tf-incident-pongthorn/{_model}"

dataset_id="SMartML"

data_table="new2_incident"
prediction_table="new2_result_binary_prediction_incident"


# unusedCols_unseen=['id','severity_name','imported_at','open_to_close_hour']

unusedCols_unseen=['id','severity_name','imported_at','range_open_to_close_hour']


# Get today's date
prediction_datetime=datetime.now(timezone.utc)
today_str=prediction_datetime.strftime("%Y-%m-%d")
today=datetime.strptime(today_str,"%Y-%m-%d")
print(f"Prediction at {prediction_datetime} for {today_str} ({today})")

print(model_gs_path)
print(f"Data: {data_table} and Prediction: {prediction_table}")

Prediction at 2023-08-06 13:00:01.046552+00:00 for 2023-08-06 (2023-08-06 00:00:00)
gs://demo-tf-incident-pongthorn/demo_binary_xgb_tf_model
Data: new2_incident and Prediction: new2_result_binary_prediction_incident


# Load Model

In [17]:
abc_model = tf.keras.models.load_model(model_gs_path)  
print(abc_model.summary())
# abc_model = tf.keras.models.load_model(model_local_path) 

[INFO 23-08-06 13:00:04.3141 UTC kernel.cc:1243] Loading model from path gs://demo-tf-incident-pongthorn/demo_binary_xgb_tf_model/assets/ with prefix 1a7b1c3944f24706
[INFO 23-08-06 13:00:04.8914 UTC abstract_model.cc:1311] Engine "GradientBoostedTreesQuickScorerExtended" built
[INFO 23-08-06 13:00:04.8921 UTC kernel.cc:1075] Use fast generic engine


Model: "gradient_boosted_trees_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1
Trainable params: 0
Non-trainable params: 1
_________________________________________________________________
None


# BigQuery Configuration

In [18]:
client = bigquery.Client(project=projectId)
new_data_table_id=f"{projectId}.{dataset_id}.{data_table}"
predictResult_table_id=f"{projectId}.{dataset_id}.{prediction_table}"
print(new_data_table_id)
print(predictResult_table_id)

pongthorn.SMartML.new2_incident
pongthorn.SMartML.new2_result_binary_prediction_incident


In [19]:
try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
    
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prediction_item_date", "DATE", mode="REQUIRED"),    
    bigquery.SchemaField("label_binary_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("pred_binary_severity", "INTEGER", mode="REQUIRED"),       
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") ,
    bigquery.SchemaField("model_version", "STRING", mode="REQUIRED")     
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table pongthorn.SMartML.new2_result_binary_prediction_incident already exists.


# Load data to Make Prediction

In [25]:
if int(all_prediction)==0:
    sql=f"""
    SELECT *  FROM `{new_data_table_id}` 
     WHERE DATE(imported_at) = '{today_str}'
     order by imported_at
    """
else:
    sql=f"""
    SELECT *  FROM `{new_data_table_id}` 
     order by imported_at
    """

print(sql)


query_result=client.query(sql)
df=query_result.to_dataframe()
if df.empty==True:
  print("no data to make prediction")  
  # return "no data to make prediction"
print(df.info())



    SELECT *  FROM `pongthorn.SMartML.new2_incident` 
     order by imported_at
    
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        100 non-null    Int64         
 1   severity_id               100 non-null    Int64         
 2   severity_name             100 non-null    object        
 3   sla                       100 non-null    object        
 4   product_type              100 non-null    object        
 5   brand                     100 non-null    object        
 6   service_type              100 non-null    object        
 7   incident_type             100 non-null    object        
 8   open_to_close_hour        100 non-null    float64       
 9   range_open_to_close_hour  100 non-null    object        
 10  imported_at               100 non-null    datetime64[ns]
dtyp

# Build Unseen data by removing label and others

In [27]:
unseen =df.drop(columns=unusedCols_unseen)
print(unseen.info())
unseen.tail(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   severity_id         100 non-null    Int64  
 1   sla                 100 non-null    object 
 2   product_type        100 non-null    object 
 3   brand               100 non-null    object 
 4   service_type        100 non-null    object 
 5   incident_type       100 non-null    object 
 6   open_to_close_hour  100 non-null    float64
dtypes: Int64(1), float64(1), object(5)
memory usage: 5.7+ KB
None


,severity_id,sla,product_type,brand,service_type,incident_type,open_to_close_hour
90,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.833333
91,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.500000
92,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.000000
93,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,14.500000
94,4,24x7 4Hrs Resolution Time,Software,Veeam,Request,Software,2.000000
95,4,24x7 4Hrs Resolution Time,Software,VMWare,Request,Upgrade Software,2.500000
96,4,24x7 4Hrs Resolution Time,Software,Veeam,Request,Software,4.000000
97,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,27.500000
98,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,34.000000
99,4,24x7 4Hrs Resolution Time,Access Point,Cisco,Request,General Incident,4.500000


# Convert dataframe to tensorflow dataset

In [28]:
unseen_ds= tfdf.keras.pd_dataframe_to_tf_dataset(unseen.drop(columns=['severity_id']))
print(unseen_ds)

<_PrefetchDataset element_spec={'sla': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'product_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'brand': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'service_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'incident_type': TensorSpec(shape=(None,), dtype=tf.string, name=None), 'open_to_close_hour': TensorSpec(shape=(None,), dtype=tf.float64, name=None)}>


In [29]:
predResultList=abc_model.predict(unseen_ds)
predServerityIDList=[]
for predResult in predResultList:
    # prob = tf.nn.sigmoid(predResult) # no need to convert to Signmoid
    # print(prob)
    # pred_seveirty_id= "critical" if _class==1 else "normal"
    _class= 1 if predResult[0]>=0.5 else 0  
    predServerityIDList.append(_class) #0=normal , 1=critical
    print(f"{predResult} : {_class}")

dfPred=pd.DataFrame(data=predServerityIDList,columns=["pred_binary_severity"])   


1/1 [==============================] - 0s 104ms/step
[0.40459675] : 0
[0.91025925] : 1
[0.25244436] : 0
[0.34089568] : 0
[0.9617809] : 1
[0.60632724] : 1
[0.81013334] : 1
[0.9286333] : 1
[0.88944894] : 1
[0.9286333] : 1
[0.9286333] : 1
[0.9286333] : 1
[0.9286333] : 1
[0.84635705] : 1
[0.90719855] : 1
[0.7699413] : 1
[0.9286333] : 1
[0.64559114] : 1
[0.95130616] : 1
[0.93978626] : 1
[0.87598825] : 1
[0.8502955] : 1
[0.23758069] : 0
[0.73691845] : 1
[0.60076374] : 1
[0.8000254] : 1
[0.38826528] : 0
[0.2761874] : 0
[0.45083866] : 0
[0.4864321] : 0
[0.26064032] : 0
[0.50909126] : 1
[0.49745125] : 0
[0.03467112] : 0
[0.01200945] : 0
[0.01200945] : 0
[0.01172972] : 0
[0.01987898] : 0
[0.01579809] : 0
[0.02643545] : 0
[0.03242865] : 0
[0.95745105] : 1
[0.8460066] : 1
[0.95449686] : 1
[0.96046495] : 1
[0.95745105] : 1
[0.96093214] : 1
[0.8357462] : 1
[0.9153345] : 1
[0.05288174] : 0
[0.50073874] : 1
[0.92288953] : 1
[0.9035213] : 1
[0.8831729] : 1
[0.72316307] : 1
[0.72316307] : 1
[0.8174651] 

2023-08-06 13:07:01.049980: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype string and shape [100]
	 [[{{node Placeholder/_5}}]]


# Map severity_id to label for actual value.
# Merge predicted value to main dataframe

In [30]:
def map_4to2_serverity(severity_id):
    if severity_id==1 or severity_id==2:
       return 1
    else:
       return 0 
df['label_binary_severity'] =df['severity_id'].apply(map_4to2_serverity)

dfPred
df=pd.concat([df,dfPred],axis=1)
# df

# Evaluate model and Show Metric Report

In [31]:
if is_evaluation:
    from sklearn.metrics import confusion_matrix,classification_report
    className=list(set().union(list(df['pred_binary_severity'].unique()),list(df['label_binary_severity'].unique())))
    print(className)
    actualClass=[  f'actual-{x}' for x in  className]
    predictedlClass=[  f'pred-{x}' for x in className]
    y_true=list(df['label_binary_severity'])
    y_pred=list(df['pred_binary_severity'])
    cnf_matrix = confusion_matrix(y_true,y_pred)
    cnf_matrix

    # #index=actual , column=prediction
    cm_df = pd.DataFrame(cnf_matrix,
                         index = actualClass, 
                         columns = predictedlClass)
    print(cm_df)
    print(classification_report(y_true, y_pred, labels=className))

[0, 1]
          pred-0  pred-1
actual-0      40       9
actual-1       6      45
              precision    recall  f1-score   support

           0       0.87      0.82      0.84        49
           1       0.83      0.88      0.86        51

    accuracy                           0.85       100
   macro avg       0.85      0.85      0.85       100
weighted avg       0.85      0.85      0.85       100



# Transform data for Writing Prediction Result to BQ

In [32]:
df=df[['id','label_binary_severity','pred_binary_severity']]
df['prediction_item_date']=today
df['prediction_datetime']=datetime.now()
df['model_version']=model_version
# print(df)

# Load ata to BQ

In [33]:
def loadDataFrameToBQ():
    # WRITE_TRUNCATE , WRITE_APPEND
    try:
        if all_prediction==1: 
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
        else:
            job_config = bigquery.LoadJobConfig(write_disposition="WRITE_APPEND")

        job = client.load_table_from_dataframe(
            df, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(df), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  100 Imported bigquery successfully


In [ ]:
# return 'All incidents has been predicted completely.'